In [1]:
# import pandas as pd 
# import numpy as np 
# import matplotlib.pyplot as plt
# import datetime
# import os

# col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]

# for i in range(1, 33):
#     col_names.append(f"Sp{i}")

# files = ['Threshold Steatoda A masking 02 pm.csv', 'Threshold Steatoda A masking 10 am.csv', 'Threshold Steatoda A masking 4 am.csv', 'Threshold Steatoda A masking midnight.csv', 'Threshold Steatoda B masking 04 pm.csv', 'Threshold Steatoda B masking 10 pm.csv', 'Threshold Steatoda B masking 12 pm.csv', 'Threshold Steatoda B maskng 2am.csv']


# df = pd.read_csv(files, names=col_names, sep='\s+', header=None)
# df = df.set_index('Index')
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
# df = df[df["MonStatus"] == 1]


# month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
# df['DateM'] = df['DateM'].str[:3].map(month_map)
# df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
# df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

# df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
#                                      month=df['Date'].dt.month,
#                                      day=df['Date'].dt.day,
#                                      hour=df['Time'].dt.hour,
#                                      minute=df['Time'].dt.minute,
#                                      second=df['Time'].dt.second))

# df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

# day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

# df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

# #df.to_csv('steatodaA.csv')
# display(df)

In [2]:
# import pandas as pd
# import numpy as np

# def entrainment(data, column):
#     dflight = data[data['Light'] == 1][column]
#     dfdark = data[data['Light'] == 0][column]
    
#     lightmean = np.mean(dflight)
#     darkmean = np.mean(dfdark)
#     diff = lightmean / darkmean
    
#     return diff > 0.25

# spiders = ["Sp"+str(i) for i in range(1, 33)]

# entrainment_results = []

# for spider_column in spiders:
#     entrainment_result = entrainment(df, spider_column)  # Use your DataFrame 'df' here
#     entrainment_results.append((spider_column, entrainment_result))

# results_df = pd.DataFrame(entrainment_results, columns=['Spider', 'Entrained'])

# print(results_df)

In [3]:
# entrainment(df, "Sp9")


In [4]:
import pandas as pd
import numpy as np
from tabulate import tabulate  # Importing the tabulate module

def entrainment(data, column):
    dflight = data[data['Light'] == 1][column]
    dfdark = data[data['Light'] == 0][column]
    
    lightmean = np.mean(dflight)
    darkmean = np.mean(dfdark)
    diff = lightmean / darkmean
    
    return diff > 0.25

col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]

for i in range(1, 33):
    col_names.append(f"Sp{i}")

files = [
    'Steatoda A masking 02 pm.txt', 'Steatoda A masking 10 am.txt',
    'Steatoda A masking 4 am.txt', 'Steatoda A masking midnight.txt',
    'Steatoda B masking 04 pm.txt', 'Steatoda B masking 10 pm.txt',
    'Steatoda B masking 12 pm.txt', 'Steatoda B maskng 2am.txt'
]

dataframes = {}  # Dictionary to store DataFrames for each file

for file in files:
    df = pd.read_csv(file, names=col_names, sep='\s+')  # Assuming files are space-separated TXT files
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    dataframes[file] = df  # Store DataFrame in the dictionary with the file name as key

spiders = ["Sp"+str(i) for i in range(1, 33)]

for file, df in dataframes.items():
    entrainment_results = []

    for spider_column in spiders:
        entrainment_result = entrainment(df, spider_column)  # Use the DataFrame for the current file
        entrainment_results.append((spider_column, entrainment_result))

    results_df = pd.DataFrame(entrainment_results, columns=['Spider', 'Entrained'])

    # Add a row to the DataFrame indicating entrainment status
    results_df.loc[-1] = ['Entrained', any(results_df['Entrained'])]

    # Reset index to maintain continuity after adding the row
    results_df.reset_index(drop=True, inplace=True)

    print(f"Results for {file}:")
    print(tabulate(results_df, headers='keys', tablefmt='fancy_grid'))  # Using tabulate to print the DataFrame in a nicer format
    
    # Saving DataFrame to CSV file
    file_name = file.replace('.txt', '_entrainment.csv')  # Creating the file name
    results_df.to_csv(file_name, index=False)


/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean


Results for Steatoda A masking 02 pm.txt:
╒════╤═══════════╤═════════════╕
│    │ Spider    │ Entrained   │
╞════╪═══════════╪═════════════╡
│  0 │ Sp1       │ False       │
├────┼───────────┼─────────────┤
│  1 │ Sp2       │ False       │
├────┼───────────┼─────────────┤
│  2 │ Sp3       │ True        │
├────┼───────────┼─────────────┤
│  3 │ Sp4       │ True        │
├────┼───────────┼─────────────┤
│  4 │ Sp5       │ True        │
├────┼───────────┼─────────────┤
│  5 │ Sp6       │ False       │
├────┼───────────┼─────────────┤
│  6 │ Sp7       │ True        │
├────┼───────────┼─────────────┤
│  7 │ Sp8       │ False       │
├────┼───────────┼─────────────┤
│  8 │ Sp9       │ True        │
├────┼───────────┼─────────────┤
│  9 │ Sp10      │ True        │
├────┼───────────┼─────────────┤
│ 10 │ Sp11      │ True        │
├────┼───────────┼─────────────┤
│ 11 │ Sp12      │ True        │
├────┼───────────┼─────────────┤
│ 12 │ Sp13      │ True        │
├────┼───────────┼─────────────┤
│

/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
/tmp/ipykernel_1302/2642108933.py:11: RuntimeWarning: invalid value encountered in scalar divide
  diff = lightmean / darkmean
